# Assigning Values to SysML Features

This notebook walks through the model modification methods for creating feature reference expressions and literal expressions to assign one or more values to the Features (usages) within a SysML model once the value has been determined by user entry or a solver.

In [ ]:
import json
from importlib import resources as lib_resources
import pymbe.api as pm

from pymbe.model import Element

from uuid import uuid4

from pymbe.query.metamodel_navigator import \
    get_effective_basic_name, get_effective_lower_multiplicity, get_effective_upper_multiplicity, get_most_specific_feature_type

from pymbe.model_modification import \
    build_from_classifier_pattern, build_from_feature_pattern, assign_value_by_literal_expression, assign_multiple_values_with_fre

from pymbe.text_concrete_syntax import serialize_sysml_package

## Load Libraries

Load up the core SysML and KerML libraries to reference against our model.

In [ ]:
library_model = None

with lib_resources.path("pymbe.static_data", "SystemsLibrary.json") as lib_data1:
    with lib_resources.path("pymbe.static_data", "KernelLibraryExpanded.json") as lib_data2:
            library_model = pm.Model.load_from_mult_post_files([lib_data1, lib_data2])

## Create empty model and reference

Have a new model that includes references back to the collective libraries so it can be built with SysML V2.

In [ ]:
empty_model = pm.Model(elements={})

package_model_namespace_data = {
    "aliasIds": [],
    "isImpliedIncluded": False,
    "@type": "Namespace",
    "@id": str(uuid4()),
    "ownedRelationship": [],
}

package_model_data = {
    "name": "SysML Value Model",
    "declaredName": "SysML Value Model",
    "isLibraryElement": False,
    "filterCondition": [],
    "ownedElement": [],
    "owner": {},
    "@type": "Package",
    "@id": str(uuid4()),
    "ownedRelationship": [],
}

new_ns = Element.new(data=package_model_namespace_data, model=empty_model)

new_package = Element.new(data=package_model_data, model=empty_model)

empty_model.reference_other_model(library_model)

In [ ]:
empty_model

In [ ]:
empty_model.ownedElement[1]

## Get Library Elements

Retrieve the library elements from which the rest of this example model will be built.

In [ ]:
part_ns = [
        library_model_ns
        for library_model_ns in library_model.ownedElement
        if library_model_ns.throughOwningMembership[0].declaredName == "Parts"
    ][0]

In [ ]:
lib_part = part_ns.throughOwningMembership[0].throughOwningMembership[2]
lib_part

In [ ]:
bf_ns = [
        library_model_ns
        for library_model_ns in library_model.ownedElement
        if library_model_ns.throughOwningMembership[0].declaredName == "BaseFunctions"
    ][0]

In [ ]:
sep_function = bf_ns.throughOwningMembership[0].throughOwningMembership[8]
sep_function

In [ ]:
new_car = build_from_classifier_pattern(
    owner=new_package,
    name="Car",
    model=empty_model,
    metatype="PartDefinition",
    superclasses=[lib_part],
    specific_fields={"ownedRelationship": []},
)

In [ ]:
new_wheel_def = build_from_classifier_pattern(
    owner=new_package,
    name="Wheel",
    model=empty_model,
    metatype="PartDefinition",
    superclasses=[lib_part],
    specific_fields={"ownedRelationship": []},
)

In [ ]:
new_wheels = build_from_feature_pattern(
    owner=new_car,
    name="wheels",
    model=empty_model,
    specific_fields={"ownedRelationship": []},
    feature_type=new_wheel_def,
    direction="",
    metatype="PartUsage",
    connector_end=False,
)

In [ ]:
wheel_diameter = build_from_feature_pattern(
    owner=new_wheel_def,
    name="diameter",
    model=empty_model,
    specific_fields={"ownedRelationship": []},
    feature_type=None,
    direction="",
    metatype="AttributeUsage",
    connector_end=False,
)

Add a specific value to the diameter of the wheel type.

In [ ]:
assign_value_by_literal_expression(
    target_feature=wheel_diameter,
    value_to_assign=16.5,
    model=empty_model
)

This is an example of what the model may look like before individual wheels are assigned values.

In [ ]:
print(serialize_sysml_package(empty_model.ownedElement[1]))

In [ ]:
wheel1 = build_from_feature_pattern(
    owner=new_car,
    name="wheel1",
    model=empty_model,
    specific_fields={"ownedRelationship": []},
    feature_type=new_wheel_def,
    direction="",
    metatype="PartUsage",
    connector_end=False,
)

In [ ]:
wheel2 = build_from_feature_pattern(
    owner=new_car,
    name="wheel2",
    model=empty_model,
    specific_fields={"ownedRelationship": []},
    feature_type=new_wheel_def,
    direction="",
    metatype="PartUsage",
    connector_end=False,
)

In [ ]:
wheel3 = build_from_feature_pattern(
    owner=new_car,
    name="wheel3",
    model=empty_model,
    specific_fields={"ownedRelationship": []},
    feature_type=new_wheel_def,
    direction="",
    metatype="PartUsage",
    connector_end=False,
)

In [ ]:
wheel4 = build_from_feature_pattern(
    owner=new_car,
    name="wheel4",
    model=empty_model,
    specific_fields={"ownedRelationship": []},
    feature_type=new_wheel_def,
    direction="",
    metatype="PartUsage",
    connector_end=False,
)

In [ ]:
assign_multiple_values_with_fre(
    type_to_apply_pattern_on=new_wheels,
    model=empty_model,
    features_to_reference=[wheel1, wheel2, wheel3, wheel4],
    seperator_function=sep_function
)

In [ ]:
print(serialize_sysml_package(empty_model.ownedElement[1]))